# **15.4 Polymorphism**

Polymorphism means "many forms" — the ability to treat different types of objects through a common interface. A battle system can accept any Pokemon and call `.attack()`, whether it's a Pikachu, Charizard, or Blastoise — each implements attack differently. In this lesson you'll learn how polymorphism enables flexible, extensible code through method overriding, duck typing, and abstract base classes.

---

## **Polymorphism Through Inheritance**

When multiple classes override the same method, you can call that method on any instance without knowing its specific type.

In [ ]:
class Pokemon:
    def __init__(self, name, level):
        self.name = name
        self.level = level
    
    def attack(self):
        return f"{self.name} uses basic attack!"

class ElectricPokemon(Pokemon):
    def attack(self):  # Override
        return f"{self.name} uses Thunderbolt! ⚡"

class FirePokemon(Pokemon):
    def attack(self):  # Override
        return f"{self.name} uses Flamethrower! 🔥"

class WaterPokemon(Pokemon):
    def attack(self):  # Override
        return f"{self.name} uses Hydro Pump! 💧"

# Create different types
team = [
    ElectricPokemon("Pikachu", 25),
    FirePokemon("Charizard", 36),
    WaterPokemon("Blastoise", 36),
]

# Polymorphism — call same method on different types
print("Battle sequence:")
for pokemon in team:
    print(f"  {pokemon.attack()}")  # Each responds differently!

---

## **Duck Typing: "If it walks like a duck..."**

Python doesn't require inheritance for polymorphism. If an object has the right methods, it works — no matter its class. This is called "duck typing."

In [ ]:
# No inheritance — just matching interfaces
class Pikachu:
    def attack(self):
        return "Pikachu uses Thunderbolt!"

class Charizard:
    def attack(self):
        return "Charizard uses Flamethrower!"

class Robot:  # Not a Pokemon!
    def attack(self):
        return "Robot fires laser!"

# Polymorphic function — works with anything that has .attack()
def execute_turn(combatant):
    """Works with any object that has an attack() method."""
    return combatant.attack()

fighters = [Pikachu(), Charizard(), Robot()]

for fighter in fighters:
    print(execute_turn(fighter))  # Works for all!

---

## **Common Interface Pattern**

Design classes to share method names, making them interchangeable in your code.

In [ ]:
class Pokemon:
    def __init__(self, name, hp):
        self.name = name
        self.hp = hp
    
    def take_turn(self, opponent):
        """Common interface — each Pokemon implements differently."""
        raise NotImplementedError("Subclass must implement take_turn")

class AggressivePokemon(Pokemon):
    def take_turn(self, opponent):
        damage = 20
        opponent.hp -= damage
        return f"{self.name} attacks aggressively for {damage} damage!"

class DefensivePokemon(Pokemon):
    def take_turn(self, opponent):
        self.hp += 10
        return f"{self.name} heals 10 HP defensively!"

class BalancedPokemon(Pokemon):
    def take_turn(self, opponent):
        if self.hp < 30:
            self.hp += 10
            return f"{self.name} heals when low!"
        else:
            opponent.hp -= 15
            return f"{self.name} attacks for 15 damage!"

# Battle system works with any Pokemon
def simulate_battle(pokemon_list):
    dummy = Pokemon("Target", 100)
    for p in pokemon_list:
        result = p.take_turn(dummy)  # Polymorphic call
        print(f"  {result}")

team = [
    AggressivePokemon("Charizard", 78),
    DefensivePokemon("Blastoise", 79),
    BalancedPokemon("Venusaur", 80),
]

print("Battle simulation:")
simulate_battle(team)

---

## **Operator Overloading (Polymorphic Operators)**

Magic methods like `__eq__`, `__lt__`, `__add__` let you use operators polymorphically.

In [ ]:
class Pokemon:
    def __init__(self, name, level, power):
        self.name = name
        self.level = level
        self.power = power
    
    def __eq__(self, other):
        """Equal if same power."""
        return self.power == other.power
    
    def __lt__(self, other):
        """Less than if lower power."""
        return self.power < other.power
    
    def __str__(self):
        return f"{self.name} (Power: {self.power})"

pikachu = Pokemon("Pikachu", 25, 100)
charizard = Pokemon("Charizard", 36, 150)
raichu = Pokemon("Raichu", 30, 100)

# Polymorphic comparison operators
print(f"{pikachu} == {raichu}: {pikachu == raichu}")
print(f"{pikachu} < {charizard}: {pikachu < charizard}")

# Works with sorted(), max(), etc.
team = [pikachu, charizard, raichu]
strongest = max(team)  # Uses __lt__ for comparison
print(f"\nStrongest: {strongest}")

---

## **isinstance() for Type Checking**

Sometimes you need to check types before acting polymorphically.

In [ ]:
class Pokemon:
    def __init__(self, name):
        self.name = name

class ElectricPokemon(Pokemon):
    pass

class WaterPokemon(Pokemon):
    pass

def calculate_damage(attacker, defender, base_damage):
    """Apply type effectiveness."""
    damage = base_damage
    
    # Type effectiveness
    if isinstance(attacker, ElectricPokemon) and isinstance(defender, WaterPokemon):
        damage *= 2
        print("  It's super effective!")
    
    return damage

pikachu = ElectricPokemon("Pikachu")
blastoise = WaterPokemon("Blastoise")

damage = calculate_damage(pikachu, blastoise, 50)
print(f"Damage: {damage}")

---

## **Abstract Base Classes (ABC)**

Use `abc` module to define interfaces that subclasses must implement.

In [ ]:
from abc import ABC, abstractmethod

class Pokemon(ABC):  # Abstract base class
    def __init__(self, name, level):
        self.name = name
        self.level = level
    
    @abstractmethod
    def attack(self):
        """Must be implemented by subclasses."""
        pass
    
    @abstractmethod
    def special_ability(self):
        """Must be implemented by subclasses."""
        pass

# This will error — can't instantiate abstract class
# p = Pokemon("Test", 1)  # TypeError!

class ElectricPokemon(Pokemon):
    def attack(self):
        return f"{self.name} uses Thunderbolt!"
    
    def special_ability(self):
        return f"{self.name} generates static charge!"

# Must implement ALL abstract methods
pikachu = ElectricPokemon("Pikachu", 25)
print(pikachu.attack())
print(pikachu.special_ability())

---

## **Practical: Polymorphic Battle System**

In [ ]:
from abc import ABC, abstractmethod
import random

class Battler(ABC):
    """Abstract base for anything that can battle."""
    
    @abstractmethod
    def attack(self, opponent):
        pass
    
    @abstractmethod
    def take_damage(self, amount):
        pass
    
    @abstractmethod
    def is_defeated(self):
        pass

class Pokemon(Battler):
    def __init__(self, name, hp, attack_power):
        self.name = name
        self.hp = hp
        self.max_hp = hp
        self.attack_power = attack_power
    
    def attack(self, opponent):
        damage = random.randint(int(self.attack_power * 0.8), self.attack_power)
        opponent.take_damage(damage)
        return f"{self.name} attacks for {damage} damage!"
    
    def take_damage(self, amount):
        self.hp = max(0, self.hp - amount)
    
    def is_defeated(self):
        return self.hp == 0
    
    def __str__(self):
        return f"{self.name} HP:{self.hp}/{self.max_hp}"

class Trainer(Battler):
    """Trainers can also battle!"""
    
    def __init__(self, name, pokemon_team):
        self.name = name
        self.team = pokemon_team
        self.current = 0
    
    def attack(self, opponent):
        if self.is_defeated():
            return f"{self.name} has no Pokemon left!"
        
        active = self.team[self.current]
        result = active.attack(opponent)
        return f"{self.name}'s {result}"
    
    def take_damage(self, amount):
        if not self.is_defeated():
            self.team[self.current].take_damage(amount)
            
            if self.team[self.current].is_defeated():
                print(f"  {self.team[self.current].name} fainted!")
                self.current += 1
                if self.current < len(self.team):
                    print(f"  {self.name} sends out {self.team[self.current].name}!")
    
    def is_defeated(self):
        return all(p.is_defeated() for p in self.team)
    
    def __str__(self):
        active = self.team[self.current] if not self.is_defeated() else None
        if active:
            return f"{self.name} ({active})"
        return f"{self.name} (defeated)"

def battle(battler1, battler2):
    """Polymorphic battle — works with any Battler."""
    print(f"Battle: {battler1.name} vs {battler2.name}\n")
    
    turn = 0
    while not battler1.is_defeated() and not battler2.is_defeated():
        turn += 1
        print(f"Turn {turn}:")
        
        # First attacks
        print(f"  {battler1.attack(battler2)}")
        if battler2.is_defeated():
            break
        
        # Second attacks
        print(f"  {battler2.attack(battler1)}")
        
        print()
    
    winner = battler1 if not battler1.is_defeated() else battler2
    print(f"\n{winner.name} wins!")

# Test polymorphism
print("=" * 50)
print("Pokemon vs Pokemon")
print("=" * 50 + "\n")

pikachu = Pokemon("Pikachu", 35, 25)
onix = Pokemon("Onix", 70, 20)
battle(pikachu, onix)

print("\n" + "=" * 50)
print("Trainer vs Pokemon")
print("=" * 50 + "\n")

ash = Trainer("Ash", [
    Pokemon("Pikachu", 35, 30),
    Pokemon("Charizard", 78, 40),
])
mewtwo = Pokemon("Mewtwo", 106, 50)
battle(ash, mewtwo)

---

## **Practice Exercises**

### **Task 1-10: See notebook for tasks**

In [ ]:
# Tasks 1-10 follow same pattern as other notebooks

---

## **Summary**

- Polymorphism = same interface, different implementations
- Override methods in subclasses for different behavior
- Duck typing: if it has the right methods, it works
- Common interfaces make objects interchangeable
- `isinstance()` for type-specific behavior
- Abstract base classes enforce interface compliance
- Operator overloading enables polymorphic operators